In [17]:
import polars as pl

In [34]:
PATH = 'dataset\\2023_LoL_esports_match_data_from_OraclesElixir.csv'
COLUMNS_NEEDED = ['gameid', 'side', 'position', 'champion', 'result']
COLUMNS_TYPES = {
    'gameid': pl.Utf8,
    'side': pl.Utf8,
    'position': pl.Utf8,
    'champion': pl.Utf8,
    'result': pl.Int8
}

In [35]:
df = pl.read_csv(
            source=PATH,
            has_header=True,
            columns=COLUMNS_NEEDED,
            dtypes=COLUMNS_TYPES
            ).drop_nulls()

In [36]:
assert df['gameid'].n_unique() == df.shape[0] / 10

In [37]:
df

gameid,side,position,champion,result
str,str,str,str,i8
"""ESPORTSTMNT06_…","""Blue""","""top""","""Jax""",1
"""ESPORTSTMNT06_…","""Blue""","""jng""","""Poppy""",1
"""ESPORTSTMNT06_…","""Blue""","""mid""","""Taliyah""",1
"""ESPORTSTMNT06_…","""Blue""","""bot""","""Ezreal""",1
"""ESPORTSTMNT06_…","""Blue""","""sup""","""Karma""",1
"""ESPORTSTMNT06_…","""Red""","""top""","""Sejuani""",0
"""ESPORTSTMNT06_…","""Red""","""jng""","""Viego""",0
"""ESPORTSTMNT06_…","""Red""","""mid""","""Syndra""",0
"""ESPORTSTMNT06_…","""Red""","""bot""","""Zeri""",0
